In [7]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
ls

 Volume in drive C is Windows
 Volume Serial Number is CAA3-57F9

 Directory of C:\Users\Jared Chung\Desktop\kaggleData\telstra

28/12/2015  11:10 AM    <DIR>          .
28/12/2015  11:10 AM    <DIR>          ..
22/09/2015  12:41 PM           604,547 event_type.csv
21/09/2015  06:57 PM         1,137,312 log_feature.csv
22/09/2015  12:41 PM           451,101 resource_type.csv
21/09/2015  06:57 PM           397,055 severity_type.csv
21/09/2015  06:57 PM           206,541 test.csv
21/09/2015  06:57 PM           151,234 train.csv
               6 File(s)      2,947,790 bytes
               2 Dir(s)  149,219,540,992 bytes free


In [4]:
cd telstra

C:\Users\Jared Chung\Desktop\kaggleData\telstra


In [8]:
train = pd.read_csv("train.csv")


In [9]:
train.head()

,id,location,fault_severity
0,14121,location 118,1
1,9320,location 91,0
2,14394,location 152,1
3,8218,location 931,1
4,14804,location 120,0
